#Ingest-Customer


In [0]:
#Ingest

df_customer= spark.read.csv("/Volumes/salesdb/internetsales/internetsalesvolume/customer_data.csv", header=True, inferSchema=True)

#Ingest-Product


In [0]:
df_product=spark.read.csv("/Volumes/salesdb/internetsales/internetsalesvolume/product_data.csv", header=True, inferSchema=True)
df_product.show()

#Ingest-Sales_Data


In [0]:
df_sales_data=spark.read.csv("/Volumes/salesdb/internetsales/internetsalesvolume/sales_data.csv", header=True, inferSchema=True)
df_sales_data.show()

#Ingest-Geography_Data


In [0]:
df_geo=spark.read.csv("/Volumes/salesdb/internetsales/internetsalesvolume/geography_data.csv", header=True, inferSchema=True)

#Cleanse


In [0]:
from pyspark.sql.functions import regexp_replace
# Cleanse stage
#df_clean = df.dropna()  # Remove rows with missing values
#df_clean = df_clean.dropDuplicates()  # Remove duplicate rows
#df_clean.show()
new_domain="bidsworld.com"
df_customer_updated = df_customer.withColumn("Email", regexp_replace("Email", "@[^.]+\\..+", f"@{new_domain}"))
df_customer_updated.show()

#DataModel

In [0]:
from pyspark.sql.functions import sum

df_model_joins = df_product.join(
    df_sales_data, df_product.ProductID == df_sales_data.ProductID
).join(df_customer_updated, df_sales_data.CustomerID == df_customer.CustomerID)

df_model_final = (
    df_model_joins.groupBy(
        df_sales_data.CustomerID,
        df_customer_updated.CustomerName,
        df_product.Category,
        df_sales_data.SaleDate,
    )
    .agg(sum(df_sales_data.Quantity * df_product.Price).alias("TotalOrderAmount"))
    .orderBy("TotalOrderAmount", ascending=False)
)
df_model_final.dropDuplicates()
display(df_model_final)

#Data Analytics

In [0]:

%python
from pyspark.sql.functions import sum

df_model_final_grouped = df_model_final.groupBy(
    df_product.Category
).agg(
    sum("TotalOrderAmount").alias("TotalOrderAmount")
).orderBy(
    "TotalOrderAmount", ascending=False
).limit(20)

display(df_model_final_grouped)


Databricks visualization. Run in Databricks to view.